## Load required Python libraries
The code will require loading the following well-known Python libaries: pandas, numpy and statsmodels

In [ ]:
## Load libraries 
import pandas as pd
import numpy as np 
import statsmodels.api as sm

## Load input data

We start by loading the input dataset containing mock average price data and other relevant country-level information. 

In [ ]:
#Load price data
data="price_data.csv"
prices=pd.read_csv(data) 
prices # Show full dataset 

In [ ]:
#Load ppp data
datappp="ppp_reg.csv"
ppp_reg=pd.read_csv(datappp) 
ppp_reg # Show full dataset 

In [ ]:
## Select the base or numeraire currency
numeraire = 'C' 
numeraire_c = 'country11' 

In [ ]:
## Prep
## Drop country-item observations without a price
prices = prices[prices['price'].notnull()]

## Dataframe with country prices
d_region=pd.get_dummies(prices['region'])

## Prepare design matrix for CPD-W
d_region=pd.get_dummies(prices['region'])
d_region.drop(numeraire, axis=1, inplace=True) #drop numeraire
d_region = d_region.add_prefix('r_') #add prefix to countries
d_item=pd.get_dummies(prices['item'],drop_first=False) #include all item dummies
d_item = d_item.add_prefix('i_') #add prefix to items
prices=pd.concat([prices,d_region,d_item],axis=1) # Concatenate the new cols

## Create empty arrays to store results
l_coef= [] # to store exp(beta_hats)
l_bh= [] # to store bh labels

prices

###  Run the CPD-W on each basic heading and store results

In [ ]:
for bh in prices.bh.unique():
    tempdf=prices[prices.bh == bh] 
    X=tempdf.loc[:, [x for x in tempdf.columns if x.startswith(('r_', 'i_'))]]
    y = np.log(tempdf['price']/tempdf['ppp_reg']) 
    wts=tempdf['imp']

    wts_cpd=sm.WLS(y, X,weights=wts)
    res=wts_cpd.fit()
    res_eparams=np.exp(res.params)
    
    print("\n","Basic Heading:", bh, "\n")
    print('Exponentiated Parameters:',"\n",
          res_eparams)
    
    l_coef.append(res_eparams)
    l_bh.append(bh)

coef = np.array(l_coef, dtype=float)
coef = np.round(coef,4) # round to 4 decimals
cols = list(X)
 #store column heads of X as a list
coef[coef == 1] = np.nan #%% replace PPPs that were exp(0)=1 with 'np.nan'

#ppp=np.array(ppp_reg.ppp_reg, dtype=float)

###  Gather and display the estimated LFs 

In [ ]:
#Create dataframe of PPP results from numpy arrays
#dimension = "# BHs" x "# coef"
df_bhppp=pd.DataFrame(data = coef, index = l_bh, columns = cols)
r_numeraire=f"r_{numeraire}"
df_bhppp.insert(0, r_numeraire, 1.000) #insert column of 1s for numeraire

df_bhppp=df_bhppp.loc[:, [x for x in df_bhppp.columns if x.startswith(('r_'))]] #subsetting to store only country level PPPs
df_bhppp.columns = df_bhppp.columns.str.replace('^r_', '') 

df_bhppp['bh'] = df_bhppp.index

df_bhppp=df_bhppp.melt(id_vars="bh",var_name="region", value_name="lf")
df_bhppp



### Merging the LF data frame with the PPP reg one

In [ ]:
ppp_regLF=pd.merge(ppp_reg, df_bhppp, how='inner', on=('bh', 'region'))
ppp_regLF['ppp_linked']=ppp_regLF['ppp_reg']*ppp_regLF['lf']
ppp_regLF = ppp_regLF.drop(['ppp_reg'], axis=1)
ppp_regLF=ppp_regLF.pivot(index="bh",
              columns="country",
              values="ppp_linked").reset_index()


ppp_regLF.set_index(ppp_regLF['bh'], drop=True, append=False, inplace=True)
ppp_regLF = ppp_regLF.drop(['bh'], axis=1)
#Column sorting function
def sorting(first_col, df):
    columns = df.columns.tolist()
    columns.remove(first_col)
    columns.insert(0,first_col)
    return df.reindex(columns, axis=1)

#Sort cols with numeraire as col1
ppp_regLF=sorting(numeraire_c,ppp_regLF)


In [ ]:
#Load basic heading expenditure values
#Should contain bh and countries with prefix c
code="bhdata_exp.csv"
df_bh=pd.read_csv(code,index_col="icp_bh")
df_bh

In [ ]:
#sort rows alphabetically 
df_bh=df_bh.sort_values('icp_bh')

print("\n","Basic Heading Expenditure Values in Local Currency Units","\n")
print(df_bh, "\n")

###  Check the basic heading PPP and basic heading expenditure matrices
Before proceeding, it is important to check that both the basic heading PPP and basic heading expenditure matrices have the same dimensions. It is also important to check that the matrix of basic heading PPPs is complete. If the dimensions of the two matrices do not match or the basic heading PPP matrix is incomplete then aggregation at higher aggregate levels is not possible using the formulas employed by the ICP. 

In [ ]:
df_bh.columns = df_bh.columns.str.replace('^c_', '') 

print("Dimensions of Matrices (No. of headings x No. of countries):","\n")
print("BH Purchasing Power Parities (PPPs)  = ",ppp_regLF.shape)
print("BH Nominal Expenditures in LCUs      = ", df_bh.shape)

###  Calculate bilateral PPPs (Laspeyres-, Paasche-, and Fisher-type)

In [ ]:
#Calculate Laspeyres bilateral PPPs 
shape = (len(df_bh.columns),len(df_bh.columns))
lp = np.zeros(shape) #square matrix: country x country
nrow= len(lp)  # gets the number of rows
ncol = len(lp[0]) #get the number of cols

for row in range(nrow):
    for col in range(ncol):
        #weighted means by looping over df rows
        lp[row][col]= np.average((ppp_regLF.iloc[:,row]/ppp_regLF.iloc[:,col]),weights=df_bh.iloc[:,col])

lp_ppp = lp
lp_ppp=pd.DataFrame(data = lp_ppp, index = df_bh.columns, columns = df_bh.columns)
lp_ppp = round(lp_ppp,3)

In [ ]:
print("\n", "Laspeyres-type bilateral PPPs:","\n")
print(lp_ppp, "\n")

In [ ]:
#Calculate Paasche bilateral PPPs 
pa = np.transpose(np.reciprocal(lp))
pa_ppp=pd.DataFrame(data = pa, index = df_bh.columns, columns = df_bh.columns)
pa_ppp = round(pa_ppp,3)

In [ ]:
print("\n", "Paasche-type bilateral PPPs:","\n")
print(pa_ppp, "\n")

In [ ]:
#Create geomean function
def nangmean(arr, axis=None):
    arr = np.asarray(arr)
    inverse_valids = 1. / np.sum(~np.isnan(arr), axis=axis)  # could be a problem for all-nan-axis
    rhs = inverse_valids * np.nansum(np.log(arr), axis=axis)
    return np.exp(rhs)

#Calculate Fisher bilateral PPPs 
fi = np.zeros(shape)
nrow=len(fi)
ncol=len(fi[0])

for row in range(nrow):
    for col in range(ncol):
        fi[row][col]= nangmean([lp[row][col],pa[row][col]])

fi_ppp=pd.DataFrame(data = fi, index = df_bh.columns, columns = df_bh.columns)
fi_ppp = round(fi_ppp,3)

In [ ]:
print("Fisher-type bilateral PPPs:","\n")
print(fi_ppp, "\n")

In [ ]:
#Calculate GEKS multilateral ppps 
##requires the earlier nangmean function 
geks = np.zeros(shape)  # zero 'country x country' matrix
nrow=len(geks)  # gets the number of rows
ncol=len(geks[0])

for row in range(nrow):
    for col in range(ncol):
        geks[row][col]= nangmean(fi[row]/fi[col])     

geks_vec = np.zeros(shape=(1,len(df_bh.columns))) # as we need a vector of ppps, not a matrix
j=len(geks_vec[0])
for col in range(j):#..one PPP per country, or col of bhexp df
    geks_vec[:,col]=nangmean(geks[col,0]/geks[0,0]) #geomean over each row, w/ each col rebased to country in col1    

geks_ppp = np.array(geks_vec)

In [ ]:
geks_ppp = pd.DataFrame(geks_ppp)
geks_ppp.columns = df_bh.columns
geks_ppp = round(geks_ppp,3)

print("\n","GEKS Multilateral PPPs:","\n")
print(geks_ppp.to_string(index=False), "\n")

## Aggregation trhoguh CAR-method

### Construct a data frame at the country level

In [ ]:
#reshaping the global GEKS data frame 
geks_ppp=geks_ppp.melt(var_name="country", value_name="geks")
geks_ppp

In [ ]:
#Preparing a dataframe with aggregate regional PPPs and total expenditures 

volshare_df = ppp_reg[ppp_reg['bh'] == 'total']

df_bhtotal=df_bh.sum().to_frame().T 
df_bhtotal=df_bhtotal.melt(var_name="country", value_name="total_exp")

volshare_df=pd.merge(volshare_df, df_bhtotal, how='inner', on='country')
volshare_df['exp_ppp']=volshare_df['total_exp']/volshare_df['ppp_reg']
volshare_df['reg_total']= volshare_df['exp_ppp'].groupby(volshare_df['region']).transform('sum')
volshare_df['volshare']=volshare_df['exp_ppp']/volshare_df['reg_total']
volshare_df

In [ ]:
#Merging the expenditure and global geks dataframe
car_df=pd.merge(volshare_df, geks_ppp, how='inner', on=('country'))
#Converting the total exp using the global geks
car_df['exp_gek']=car_df['total_exp']/car_df['geks']
#Calculating the total regional expenditure in geks adjusted units
car_df['exp_gek_reg']=car_df['exp_gek'].groupby(volshare_df['region']).transform('sum')
#Applying the regional volume share to the total expenditure and re-basingit on the numeraire PPP
car_df['exp_adj']=car_df['exp_gek_reg']*car_df['volshare']
car_df['PPPglobal']=car_df['total_exp']/car_df['exp_adj']
car_df['PPPglobal_num']=car_df['PPPglobal']/0.981471
print("\n","Global linked PPPs:","\n")
print(car_df.PPPglobal_num, "\n")